¿Es posible realizar predicciones sobre el destino de un cliente utilizando variables predictoras como el tiempo, el lugar de partida, la edad y el identificador del cliente?

En esta notebook exploramos distintos clasificadores asi como variaciones de los mismos para ver el proder predictivo que se tiene para identificar donde un usuario terminara basado en la informacion que tenemos al inicio de su viaje.

# Librerias

In [7]:
# Useful
import pandas as pd
import datetime
import numpy as np
# Data helpers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# Classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# Clustering
from sklearn.cluster import AgglomerativeClustering
# Ploting
import plotly.graph_objects as go
import plotly.express as px

# Dataset

Vamos a preparar nuestros datos para poder ser usados por los distintos clasificadores que nos proporciona Sklearn.

In [13]:
BiciDF = pd.read_csv("../data/datos_abiertos_2023_03.csv", encoding = 'latin-1')

# Parse String to Datatime
BiciDF["Inicio_del_viaje"] = pd.to_datetime(BiciDF["Inicio_del_viaje"],
               format='%Y-%m-%d %H:%M:%S')
BiciDF["Fin_del_viaje"] = pd.to_datetime(BiciDF["Fin_del_viaje"],
               format='%Y-%m-%d %H:%M:%S')

# Make the year a int instead of float
BiciDF = BiciDF.dropna()
BiciDF["Anio_de_nacimiento"] = BiciDF["Anio_de_nacimiento"].astype(int)

BiciDF.head()

,Viaje_Id,Usuario_Id,Genero,Anio_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id
0,27598660,12551,M,1987,2023-03-01 00:00:32,2023-03-01 00:11:09,80,39
1,27598661,1153034,F,1994,2023-03-01 00:00:38,2023-03-01 00:09:47,188,17
2,27598668,1566425,M,1973,2023-03-01 00:04:52,2023-03-01 00:20:43,86,4
3,27598678,556440,M,2000,2023-03-01 00:05:55,2023-03-01 00:18:36,85,85
4,27598686,344644,M,2000,2023-03-01 00:07:33,2023-03-01 00:18:54,241,198


Tenemos que hacer un parsing de los datos que no son enteros en este caso las caracteristicas que vamos a manejar que no son enteros son el Genero y el inicio del viaje. El genero es sencillo como designa un genero como 1 y al otros con el 0. Por otro lado, para el inicio de viaje una opcion seria pasar todo a UNIX time; sin embargo, esto no da mucha informacion a un clasificador y es una codificacion bastante burda.

Por la naturaleza de los datos uno puede intuir que se tienen ciertos patrones semanales e interesa mas analizar estos patrones para el clasificador que solo el UNIX time. Es decir, es posible que los tiempos de los lunes se parezcan mas que a los de otro lunes que a los del miercoles de la misma semana. Entonces, vamos tomar el *tiempos que ha pasado desde el inicio de semana*. Ademas de eso para tener una mejor agrupacion tomaremos intervalos de 15 minutos. Es decir, todos los Lunes de $0:00$ a $0:15$ seran identificados con la etiqueta $0$, los del $0:15$ asl $0:30$ con la etiqueta $1$ y asi sucesivamente. La funcion mostrada a continuacion hace justamente ese mapping de los tiempos.

In [14]:
def parse_datetime(dt):
    start_of_week = dt - datetime.timedelta(days=dt.weekday())
    start_of_week = start_of_week.replace(hour=0, minute=0, second=0)
    elapsed_time = dt - start_of_week
    minutes = elapsed_time.total_seconds()/60
    return int(minutes) // 15

parse_datetime(datetime.datetime.now())

563

En la siguiente celda dividimos nuestros datos en un conjunto de entrenamiento y uno de prueba para medir el rendimiento de los clasificadores. De igual forma hacemos los castings pertinentes y trabajamos con numpy arrays para mantener uniformidad en como trabajamos con las entradas y salidas de nuestros clasificadores.

In [24]:
def parseDF(df):
    # Make Gender a number
    df["Genero"] = df["Genero"].map(lambda x: x=='M')
    # Datetime to int
    df["Inicio_del_viaje"] = df["Inicio_del_viaje"].map(parse_datetime)
    return df

trainDF, testDF = train_test_split(BiciDF, test_size=0.2)
# Prepare train data
train_features = parseDF(trainDF[["Genero", "Anio_de_nacimiento", "Inicio_del_viaje", "Origen_Id"]]).to_numpy()
train_labels   = trainDF[["Destino_Id"]].to_numpy().flatten()

# Prepare test data
test_features = parseDF(testDF[["Genero", "Anio_de_nacimiento", "Inicio_del_viaje", "Origen_Id"]]).to_numpy()
test_labels   = testDF[["Destino_Id"]].to_numpy().flatten()

print(f"We have {len(train_labels)} samples for training and {len(test_labels)} for Testing")

C:\Users\joses\AppData\Local\Temp/ipykernel_15912/2519341705.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\joses\AppData\Local\Temp/ipykernel_15912/2519341705.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



We have 319448 samples for training and 79862 for Testing


In [23]:
def check_classifier_and_cm(classifier, name):
    train_score = classifier.score(train_features, train_labels)
    test_score  = classifier.score(test_features, test_labels)

    print(f"Training score: {train_score}")
    print(f"Test score: {test_score}")

    predicted_labels = classifier.predict(test_features)
    cm = confusion_matrix(predicted_labels, test_labels)

    fig = go.Figure(data=go.Heatmap(z=cm))
    fig.update_layout(
        title=f"Matriz de confusion ({name})", 
        xaxis_title = "Clase Predicha",
        yaxis_title = "Clase Verdadera"
    )
    fig.show()

    good_traces = cm.diagonal()
    bad_traces  = np.sum(cm, axis = 1) - good_traces
    
    fig = go.Figure(data=[
        go.Bar(name='Predicciones Correctas',   y = good_traces),
        go.Bar(name='Predicciones Incorrectas', y = bad_traces)
    ])
    fig.update_layout(
        barmode = 'group', 
        xaxis_title = "Clase Predicha",
        title = f"Score de las predicciones ({name})"
    )
    fig.show()

    bad_traces  = np.sum(cm, axis = 0) - good_traces
    fig = go.Figure(data=[
        go.Bar(name='Predicciones Correctas',   y = good_traces),
        go.Bar(name='Predicciones Incorrectas', y = bad_traces)
    ])
    fig.update_layout(
        barmode='group', 
        xaxis_title = "Clase Verdadera",
        title = f"Score para cada clase ({name})"
    )
    fig.show()

# Clasificadores

## Red Neuronal

Como primer clasificador usaremos una red neuronal. En este caso, se eligio una configuracion $(16, 32, 64)$ para los perceptrones de las capas. Lamentablemente tenemos alrededor de 370 lugares para predicir y por lo tanto el entrenamiento es algo tardado.

In [ ]:
NN = MLPClassifier(
    hidden_layer_sizes=(16, 32, 64), 
    random_state=1234, 
    verbose=True, # Print classifier progress
    #max_iter=10 # Set max iteration to get results sooner
).fit(train_features, train_labels)

check_classifier_and_cm(NN, "Red Neuronal")

## Clasificador de Vecinos mas Cercanos

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 15).fit(train_features, train_labels)
check_classifier_and_cm(KNN, "K Vecinos mas Cercanos")

## Bosque Aleatorio

In [ ]:
RandomForest = RandomForestClassifier(
    n_estimators=20,
    verbose=True
).fit(train_features, train_labels)
check_classifier_and_cm(RandomForest, "Bosque aleatorio")

## Arbol de Decisiones

In [ ]:
DecisionTree = DecisionTreeClassifier().fit(train_features, train_labels)
check_classifier_and_cm(DecisionTree, "Arbol de Decisiones")

## Analisis de Discriminante Cuadratico

In [ ]:
QDA = QuadraticDiscriminantAnalysis().fit(train_features, train_labels)
check_classifier_and_cm(QDA, "Analisis Discriminante Cuadratico")

# Agrupacion de Lugares

Uno de los problemas que tienen los clasificadores que estamos usando es que tienen muchas clases sobre las cuales predecir (sin mencionar el desbalanceo de clases que se nos pidio dejarlo asi), por esta razon el siguiente paso que se puede realizar es intentar hacer un clasificador mas sencillo. 

Podemos dividir cada una de las localizaciones en `zonas` y podemos intentar predecir a que `zona` ira cada usuario en lugar de predecir el lugar exacto de llegada. Para esto usuaremos el algoritmo de agrupamiento aglomerada para crear esta clase de zonas.

In [3]:
LocationsDF = pd.read_csv("../data/nomenclatura_2023_02.csv", encoding = 'latin-1')
zonesDF = LocationsDF[["id", "latitude", "longitude"]]
zonesDF.head()

,id,latitude,longitude
0,2,20.666378,-103.348820
1,3,20.667228,-103.366000
2,4,20.667690,-103.368252
3,5,20.691750,-103.362550
4,6,20.681151,-103.338863


In [9]:
coords = zonesDF[["latitude", "longitude"]].to_numpy()
clustering = AgglomerativeClustering(n_clusters=8).fit(coords)
zonesDF["zone"] = clustering.labels_
zonesDF.head()

C:\Users\joses\AppData\Local\Temp/ipykernel_15912/459432722.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,latitude,longitude,zone
0,2,20.666378,-103.348820,1
1,3,20.667228,-103.366000,1
2,4,20.667690,-103.368252,1
3,5,20.691750,-103.362550,6
4,6,20.681151,-103.338863,3


In [10]:
fig = px.scatter(zonesDF, x="latitude", y="longitude", color="zone")
fig.show()

In [27]:
trainDF2 = trainDF.merge(
    zonesDF, 
    left_on  = 'Destino_Id', 
    right_on = 'id', 
    how = 'left'
).drop(
    ['Destino_Id', 'id', 'latitude', 'longitude'],
    axis = 1
).rename(
    columns={'zone':'DestinoZone'}
).merge(
    zonesDF,
    left_on  = 'Origen_Id', 
    right_on = 'id', 
    how = 'left'
).drop(
    ['Origen_Id', 'id', 'latitude', 'longitude'],
    axis = 1
).rename(
    columns={'zone':'OriginalZone'}
)
trainDF2.head()

,Viaje_Id,Usuario_Id,Genero,Anio_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,DestinoZone,OriginalZone
0,27914606,363157,M,1996,2023-03-21 17:11:52,2023-03-21 17:22:39,3,6
1,27998120,2143755,M,1996,2023-03-27 10:25:18,2023-03-27 10:36:10,6,6
2,27697866,273540,M,1972,2023-03-07 11:32:34,2023-03-07 11:42:57,1,1
3,27787883,1452341,M,1996,2023-03-13 08:30:31,2023-03-13 08:40:37,3,3
4,28039808,2221101,F,1983,2023-03-29 17:41:24,2023-03-29 17:55:28,1,1


In [28]:
testDF2 = testDF.merge(
    zonesDF, 
    left_on  = 'Destino_Id', 
    right_on = 'id', 
    how = 'left'
).drop(
    ['Destino_Id', 'id', 'latitude', 'longitude'],
    axis = 1
).rename(
    columns={'zone':'DestinoZone'}
).merge(
    zonesDF,
    left_on  = 'Origen_Id', 
    right_on = 'id', 
    how = 'left'
).drop(
    ['Origen_Id', 'id', 'latitude', 'longitude'],
    axis = 1
).rename(
    columns={'zone':'OriginalZone'}
)
testDF2.head()

,Viaje_Id,Usuario_Id,Genero,Anio_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,DestinoZone,OriginalZone
0,27701026,1307223,M,2002,2023-03-07 15:06:06,2023-03-07 15:08:54,1,1
1,27717616,1303255,M,1986,2023-03-08 13:08:20,2023-03-08 13:16:17,2,2
2,27849199,2111423,M,1980,2023-03-16 14:42:28,2023-03-16 14:42:49,4,4
3,27865562,2125076,M,1996,2023-03-17 13:05:27,2023-03-17 13:11:24,1,1
4,27698645,1316675,F,1994,2023-03-07 12:35:28,2023-03-07 12:42:00,0,0


In [29]:
# Prepare train data
train_features = parseDF(
    trainDF2[["Genero", "Anio_de_nacimiento", "Inicio_del_viaje", "OriginalZone"]]
).to_numpy()
train_labels = trainDF2[["DestinoZone"]].to_numpy().flatten()

# Prepare test data
test_features = parseDF(
    testDF2[["Genero", "Anio_de_nacimiento", "Inicio_del_viaje", "OriginalZone"]]
).to_numpy()
test_labels = testDF2[["DestinoZone"]].to_numpy().flatten()

print(f"We have {len(train_labels)} samples for training and {len(test_labels)} for Testing")

C:\Users\joses\AppData\Local\Temp/ipykernel_15912/2519341705.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\joses\AppData\Local\Temp/ipykernel_15912/2519341705.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



We have 319448 samples for training and 79862 for Testing


## Red Neuronal

In [30]:
NN = MLPClassifier(
    hidden_layer_sizes=(16, 32, 64), 
    random_state=1234, 
    verbose=True, # Print classifier progress
    #max_iter=10 # Set max iteration to get results sooner
).fit(train_features, train_labels)

check_classifier_and_cm(NN, "Red Neuronal")

Iteration 1, loss = 3.98020875
Iteration 2, loss = 2.38495052
Iteration 3, loss = 2.03678112
Iteration 4, loss = 1.88365700
Iteration 5, loss = 1.71391217
Iteration 6, loss = 1.65650908
Iteration 7, loss = 1.58019058
Iteration 8, loss = 1.52997520
Iteration 9, loss = 1.46835110
Iteration 10, loss = 1.42077097


c:\Users\joses\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.



Training score: 0.5044764719140518
Test score: 0.5047582079086425


## Clasificador de Vecinos mas Cercanos

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 15).fit(train_features, train_labels)
check_classifier_and_cm(KNN, "K Vecinos mas Cercanos")

## Bosque Aleatorio

In [ ]:
RandomForest = RandomForestClassifier(
    n_estimators=20,
    verbose=True
).fit(train_features, train_labels)
check_classifier_and_cm(RandomForest, "Bosque aleatorio")

## Arbol de Decisiones

In [ ]:
DecisionTree = DecisionTreeClassifier().fit(train_features, train_labels)
check_classifier_and_cm(DecisionTree, "Arbol de Decisiones")

## Analisis de Discriminante Cuadratico

In [ ]:
QDA = QuadraticDiscriminantAnalysis().fit(train_features, train_labels)
check_classifier_and_cm(QDA, "Analisis Discriminante Cuadratico")